In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1295877,2021-05-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1295878,2021-05-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1295879,2021-05-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1295880,2021-05-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
34463,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34465,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34467,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34469,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34471,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
796006,2021-05-21,Arkansas,Arkansas,5001,2119,37.00,5000,Arkansas,AR,Arkansas,State,3017804
796008,2021-05-22,Arkansas,Arkansas,5001,2119,37.00,5000,Arkansas,AR,Arkansas,State,3017804
796010,2021-05-23,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
796012,2021-05-24,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1295877,2021-05-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1295878,2021-05-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1295879,2021-05-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1295880,2021-05-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-25') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
490,2021-05-25,Snohomish,Washington,38488,582.00,53061,WA,County,822083,4681.77,70.80
978,2021-05-25,Cook,Illinois,551609,10796.00,17031,IL,County,5150233,10710.37,209.62
1465,2021-05-25,Orange,California,271725,5056.00,6059,CA,County,3175692,8556.40,159.21
1951,2021-05-25,Maricopa,Arizona,548006,10022.00,4013,AZ,County,4485414,12217.51,223.44
2437,2021-05-25,Los Angeles,California,1239758,24201.00,6037,CA,County,10039107,12349.29,241.07
...,...,...,...,...,...,...,...,...,...,...,...
1295105,2021-05-25,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08
1295330,2021-05-25,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1295524,2021-05-25,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1295714,2021-05-25,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
490,2021-05-25,Snohomish,Washington,38488,582.00,53061,WA,County,822083,4681.77,70.80,70.80,4681.77
978,2021-05-25,Cook,Illinois,551609,10796.00,17031,IL,County,5150233,10710.37,209.62,209.62,10710.37
1465,2021-05-25,Orange,California,271725,5056.00,6059,CA,County,3175692,8556.40,159.21,159.21,8556.40
1951,2021-05-25,Maricopa,Arizona,548006,10022.00,4013,AZ,County,4485414,12217.51,223.44,223.44,12217.51
2437,2021-05-25,Los Angeles,California,1239758,24201.00,6037,CA,County,10039107,12349.29,241.07,241.07,12349.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295105,2021-05-25,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08,75.08,2552.55
1295330,2021-05-25,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1295524,2021-05-25,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1295714,2021-05-25,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
832389,2021-05-25,Hale,Alabama,2247,78.00,1065,AL,County,14651,15336.84,532.39,532.39,15336.84,1
906159,2021-05-25,Clarke,Alabama,3518,61.00,1025,AL,County,23622,14892.90,258.23,258.23,14892.90,2
651081,2021-05-25,Lowndes,Alabama,1412,53.00,1085,AL,County,9726,14517.79,544.93,544.93,14517.79,3
577216,2021-05-25,Etowah,Alabama,13963,357.00,1055,AL,County,102268,13653.34,349.08,349.08,13653.34,4
484266,2021-05-25,Franklin,Alabama,4270,82.00,1059,AL,County,31362,13615.20,261.46,261.46,13615.20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227309,2021-05-25,Platte,Wyoming,643,11.00,56031,WY,County,8393,7661.15,131.06,131.06,7661.15,19
794321,2021-05-25,Converse,Wyoming,1012,17.00,56009,WY,County,13822,7321.66,122.99,122.99,7321.66,20
987866,2021-05-25,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21
1099732,2021-05-25,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
651081,2021-05-25,Lowndes,Alabama,1412,53.00,1085,AL,County,9726,14517.79,544.93,544.93,14517.79,3,1
832389,2021-05-25,Hale,Alabama,2247,78.00,1065,AL,County,14651,15336.84,532.39,532.39,15336.84,1,2
276675,2021-05-25,Walker,Alabama,7278,279.00,1127,AL,County,63521,11457.63,439.22,439.22,11457.63,28,3
575508,2021-05-25,Clay,Alabama,1578,57.00,1027,AL,County,13235,11922.93,430.68,430.68,11922.93,20,4
702198,2021-05-25,Greene,Alabama,927,34.00,1063,AL,County,8111,11428.92,419.18,419.18,11428.92,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700497,2021-05-25,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
963204,2021-05-25,Uinta,Wyoming,2261,13.00,56041,WY,County,20226,11178.68,64.27,64.27,11178.68,5,20
987866,2021-05-25,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21,21
648525,2021-05-25,Albany,Wyoming,4494,15.00,56001,WY,County,38880,11558.64,38.58,38.58,11558.64,4,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4681.77,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4681.77,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4681.77,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4681.77,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4681.77,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292846,2021-05-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1292847,2021-05-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1292848,2021-05-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1292849,2021-05-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
828937,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,1.00
828938,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,0.00
828939,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,0.00
828940,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15336.84,2,1,0.00
828941,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15336.84,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004430,2021-05-21,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5801.69,6,23,3.00
1004431,2021-05-22,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5801.69,6,23,0.00
1004432,2021-05-23,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5801.69,6,23,0.00
1004433,2021-05-24,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5801.69,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
648473,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14517.79,1,3,1.00,0.00
648474,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14517.79,1,3,0.00,0.00
648475,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14517.79,1,3,0.00,0.00
648476,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14517.79,1,3,0.00,0.00
648477,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14517.79,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273616,2021-05-21,Teton,Wyoming,3776,9.00,56039,WY,County,23464,16092.74,38.36,38.36,16109.79,23,1,1.00,0.00
273617,2021-05-22,Teton,Wyoming,3776,9.00,56039,WY,County,23464,16092.74,38.36,38.36,16109.79,23,1,0.00,0.00
273618,2021-05-23,Teton,Wyoming,3776,9.00,56039,WY,County,23464,16092.74,38.36,38.36,16109.79,23,1,0.00,0.00
273619,2021-05-24,Teton,Wyoming,3780,9.00,56039,WY,County,23464,16109.79,38.36,38.36,16109.79,23,1,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-25') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
330335,2021-05-25,Imperial,California,28575,735.00,6025,CA,County,181215,15768.56,405.60,405.60,15768.56,1,2,1.00,0.00
2437,2021-05-25,Los Angeles,California,1239758,24201.00,6037,CA,County,10039107,12349.29,241.07,241.07,12349.29,2,5,174.00,4.00
164387,2021-05-25,San Bernardino,California,297688,4747.00,6071,CA,County,2180085,13654.88,217.74,217.74,13654.88,3,4,73.00,0.00
578876,2021-05-25,Inyo,California,1429,38.00,6027,CA,County,18039,7921.73,210.65,210.65,7921.73,4,25,0.00,0.00
73461,2021-05-25,Stanislaus,California,62647,1066.00,6099,CA,County,550660,11376.71,193.59,193.59,11376.71,5,9,35.00,2.00
34017,2021-05-25,Riverside,California,300476,4608.00,6065,CA,County,2470546,12162.33,186.52,186.52,12162.33,6,7,155.00,4.00
58435,2021-05-25,San Joaquin,California,73717,1419.00,6077,CA,County,762148,9672.27,186.18,186.18,9672.27,7,17,100.00,16.00
91098,2021-05-25,Tulare,California,50134,842.00,6107,CA,County,466195,10753.87,180.61,180.61,10753.87,8,10,3.00,0.00
33127,2021-05-25,Fresno,California,102289,1698.00,6019,CA,County,999101,10238.10,169.95,169.95,10238.10,9,14,19.00,0.00
524537,2021-05-25,Merced,California,32093,468.00,6047,CA,County,277680,11557.55,168.54,168.54,11557.55,10,8,11.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1226860,2021-05-25,Lassen,California,5721,24.00,6035,CA,County,30573,18712.59,78.50,78.50,18712.59,35,1,0.00,0.00
330335,2021-05-25,Imperial,California,28575,735.00,6025,CA,County,181215,15768.56,405.60,405.60,15768.56,1,2,1.00,0.00
702140,2021-05-25,Kings,California,23042,247.00,6031,CA,County,152940,15066.04,161.50,161.50,15066.04,11,3,4.00,0.00
164387,2021-05-25,San Bernardino,California,297688,4747.00,6071,CA,County,2180085,13654.88,217.74,217.74,13654.88,3,4,73.00,0.00
2437,2021-05-25,Los Angeles,California,1239758,24201.00,6037,CA,County,10039107,12349.29,241.07,241.07,12349.29,2,5,174.00,4.00
206721,2021-05-25,Kern,California,109738,1385.00,6029,CA,County,900202,12190.38,153.85,153.85,12190.38,14,6,62.00,2.00
34017,2021-05-25,Riverside,California,300476,4608.00,6065,CA,County,2470546,12162.33,186.52,186.52,12162.33,6,7,155.00,4.00
524537,2021-05-25,Merced,California,32093,468.00,6047,CA,County,277680,11557.55,168.54,168.54,11557.55,10,8,11.00,0.00
73461,2021-05-25,Stanislaus,California,62647,1066.00,6099,CA,County,550660,11376.71,193.59,193.59,11376.71,5,9,35.00,2.00
91098,2021-05-25,Tulare,California,50134,842.00,6107,CA,County,466195,10753.87,180.61,180.61,10753.87,8,10,3.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
330335,2021-05-25,Imperial,California,28575,735.00,6025,CA,County,181215,15768.56,405.60,405.60,15768.56,1,2,1.00,0.00
2437,2021-05-25,Los Angeles,California,1239758,24201.00,6037,CA,County,10039107,12349.29,241.07,241.07,12349.29,2,5,174.00,4.00
164387,2021-05-25,San Bernardino,California,297688,4747.00,6071,CA,County,2180085,13654.88,217.74,217.74,13654.88,3,4,73.00,0.00
578876,2021-05-25,Inyo,California,1429,38.00,6027,CA,County,18039,7921.73,210.65,210.65,7921.73,4,25,0.00,0.00
73461,2021-05-25,Stanislaus,California,62647,1066.00,6099,CA,County,550660,11376.71,193.59,193.59,11376.71,5,9,35.00,2.00
34017,2021-05-25,Riverside,California,300476,4608.00,6065,CA,County,2470546,12162.33,186.52,186.52,12162.33,6,7,155.00,4.00
58435,2021-05-25,San Joaquin,California,73717,1419.00,6077,CA,County,762148,9672.27,186.18,186.18,9672.27,7,17,100.00,16.00
91098,2021-05-25,Tulare,California,50134,842.00,6107,CA,County,466195,10753.87,180.61,180.61,10753.87,8,10,3.00,0.00
33127,2021-05-25,Fresno,California,102289,1698.00,6019,CA,County,999101,10238.10,169.95,169.95,10238.10,9,14,19.00,0.00
524537,2021-05-25,Merced,California,32093,468.00,6047,CA,County,277680,11557.55,168.54,168.54,11557.55,10,8,11.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15768.56,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4332,05/21/21,Lassen,5720,24.00,18709.32,78.50,78.50,18712.59,35,1,0.00,0.00
4333,05/22/21,Lassen,5721,24.00,18712.59,78.50,78.50,18712.59,35,1,1.00,0.00
4334,05/23/21,Lassen,5721,24.00,18712.59,78.50,78.50,18712.59,35,1,0.00,0.00
4335,05/24/21,Lassen,5721,24.00,18712.59,78.50,78.50,18712.59,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15768.56,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15768.56,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4418,05/21/21,Merced,32028,468.00,11534.14,168.54,168.54,11557.55,10,8,23.00,0.00
4419,05/22/21,Merced,32028,468.00,11534.14,168.54,168.54,11557.55,10,8,0.00,0.00
4420,05/23/21,Merced,32028,468.00,11534.14,168.54,168.54,11557.55,10,8,0.00,0.00
4421,05/24/21,Merced,32082,468.00,11553.59,168.54,168.54,11557.55,10,8,54.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)